In [440]:
import pandas as pd
import os
import sys
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [453]:
df = pd.read_csv("lens-data-2.csv") 
df.drop(["#"], axis=1,inplace=True)
df.columns= df.columns.str.lower()
df1 = df[df['inventors'].notnull()]
maindf = df1[df1['applicants'].notnull()]
maindf.reset_index(inplace=True)
maindf.head()

,index,jurisdiction,kind,display key,lens id,publication date,publication year,application number,application date,earliest priority date,title,applicants,inventors,owners,url,legal status
0,0,US,A1,US 2017/0147742 A1,046-338-410-657-855,2017-05-25,2017,US 201515321739 A,2015-06-19,2014-06-27,SYSTEMS AND METHODS FOR SYNTHETIC BIOLOGY DESI...,UNIV NANYANG TECH,JAYARAMAN PREMKUMAR;;POH CHUEH LOO;;WANG HUI JUAN,NANYANG TECHNOLOGICAL UNIVERSITY (2015-10-29),https://lens.org/046-338-410-657-855,DISCONTINUED
1,1,US,B2,US 10607716 B2,068-534-912-401-872,2020-03-31,2020,US 201615064206 A,2016-03-08,2011-06-06,Synthetic biology tools,UNIV CALIFORNIA,LOU CHUNBO;;MOON TAE SEOK;;RHODIUS VIRGIL;;STA...,THE REGENTS OF THE UNIVERSITY OF CALIFORNIA (2...,https://lens.org/068-534-912-401-872,ACTIVE
2,2,US,A1,US 2009/0061520 A1,075-918-668-399-614,2009-03-05,2009,US 93516407 A,2007-11-05,2006-11-03,Synthetic Biology Vectors,UNIV MICHIGAN,WOOLF PETER;;NINFA ALEX;;MAYO AVRAHAM E;;SELIN...,THE REGENTS OF THE UNIVERSITY OF MICHIGAN (200...,https://lens.org/075-918-668-399-614,INACTIVE
3,3,WO,A2,WO 2021/035115 A2,009-086-590-277-234,2021-02-25,2021,US 2020/0047330 W,2020-08-21,2019-08-21,GROWING PROGRAMMABLE ENZYME-FUNCTIONALIZED AND...,MASSACHUSETTS INST TECHNOLOGY;;IMP COLLEGE LONDON,LU TIMOTHY;;TANG TZU-CHIEH;;OTT WOLFGANG;;ELLI...,NaN,https://lens.org/009-086-590-277-234,PENDING
4,4,US,A1,US 2021/0158197 A1,038-794-758-325-674,2021-05-27,2021,US 202017104817 A,2020-11-25,2019-11-26,BIOLOGY EXPERIMENT DESIGNS,UNIV CALIFORNIA,COSTELLO ZACHARY;;RADIVOJEVIC TIJANA;;MARTIN H...,THE REGENTS OF THE UNIVERSITY OF CALIFORNIA (2...,https://lens.org/038-794-758-325-674,PENDING


In [442]:
def make_dict(df, column):
    finaldict = {}
    for index, url in enumerate(df['url']):
        finaldict[url] = df[column][index].split(';;')
    return finaldict


In [443]:
def make_list(df, column):
    finallist = []
    for i in df[column]:
            finallist.append(i.split(';;'))
    return finallist

In [454]:
#dictionary of lens patent link, then inventory names
maindict = make_dict(maindf,'inventors')

In [455]:
#list of inventors related to a patent
inventor_list_init = make_list(maindf,'inventors')

In [446]:
def uniquelist(list):
    flist = []
    for l in list:
        for i in l:
            if i in flist:
                pass
            else:
                flist.append(i)
    return flist

In [458]:
#list of all unique inventors
inventorlist = uniquelist(make_list(maindf,'inventors'))
inventorlist[0:10]

['JAYARAMAN PREMKUMAR',
 'POH CHUEH LOO',
 'WANG HUI JUAN',
 'LOU CHUNBO',
 'MOON TAE SEOK',
 'RHODIUS VIRGIL',
 'STANTON BRYNNE',
 'TAMSIR ALVIN',
 'TEMME KARSTEN',
 'VOIGT CHRIS']

In [449]:
newdf = pd.DataFrame({'jurisdiction' : maindf['jurisdiction'],
                      'publication year' : maindf['publication year'],
                      'title' : maindf['title']
                      })
newdf.head()

,jurisdiction,publication year,title
0,US,2017,SYSTEMS AND METHODS FOR SYNTHETIC BIOLOGY DESI...
1,US,2020,Synthetic biology tools
2,US,2009,Synthetic Biology Vectors
3,WO,2021,GROWING PROGRAMMABLE ENZYME-FUNCTIONALIZED AND...
4,US,2021,BIOLOGY EXPERIMENT DESIGNS


In [450]:

def datacol(prof):
    #go to profile input
    profile_key = driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/nav/ul/li[3]/a")
    profile_key.click()

    #input name
    input = driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/form/div/div[1]/label/input")
    input.send_keys(prof)

    #click search
    searchb = driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/form/div/div[1]/button").click()
    try: searchb
    except: 
        driver.get(url)
        return(['no information found'])
    
    time.sleep(5)

    try:
        driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/form/div/div[2]/div[1]")
        pass
    except:
        driver.get(url)
        return(['no information found'])
    
    #new link
    try:
        link = driver.find_element(By.XPATH, "/html/body/main/header/article/div[1]/div/div/form/div/div[2]/div[1]/div/a")
    except:
        driver.get(url)
        return(['no information found'])
    linkn = link.get_attribute('href')
    driver.get(linkn)

    #collecting data---

    #waiting for data to load
    time.sleep(5)
    def waiting():
        if driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[1]/a/strong").text == '-':
            time.sleep(2)
            waiting()
        else:
            pass
    waiting()

    #data collection points
    nworks = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[1]/a/strong").text
    colab_ratio = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[3]/a/strong").text
    lens_h_index = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[4]/a/strong").text
    scholar_cites = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[6]/a/strong").text
    patent_cites = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/ul/li[7]/a/strong").text
    fields = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[1]/div[2]/div").text
    lensname = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[1]/div/div[2]/div[1]/h1").text

    try: workcheck = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[3]/div[2]/div/div[1]").text
    except: workcheck = 'N/A'

    if workcheck == 'N/A': placeofwork = 'no current information'
    else:
        if 'Present' in workcheck: placeofwork = driver.find_element(By.XPATH, "/html/body/div[3]/main/section/header/div[1]/div/div/div[2]/div[3]/div[2]/div[1]/div[2]/strong").text
        else: placeofwork = 'no current information'

    driver.get(url)
    return({'name_lens' : lensname, 
            'lens_link' : linkn, 
            'number_of_works' : nworks, 
            'collaboration_ratio' : colab_ratio, 
            'lens_h_index' : lens_h_index, 
            'scholar_article_cites' : scholar_cites, 
            'patent_cites' : patent_cites, 
            'associated_fields' : fields, 
            'current_place_of_work' : placeofwork})





In [451]:
def makedict(researcherl):
    datadict = {}
    for r in researcherl:
        datadict[r] = datacol(r)
    return datadict
    

In [452]:
url = 'https://www.lens.org/'
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

first_thousand_researcher_data = makedict(inventorlist[0:999])

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DC8893+48451]
	(No symbol) [0x00D5B8A1]
	(No symbol) [0x00C65058]
	(No symbol) [0x00C4D073]
	(No symbol) [0x00CADEBB]
	(No symbol) [0x00CBBFD3]
	(No symbol) [0x00CAA0B6]
	(No symbol) [0x00C87E08]
	(No symbol) [0x00C88F2D]
	GetHandleVerifier [0x01028E3A+2540266]
	GetHandleVerifier [0x01068959+2801161]
	GetHandleVerifier [0x0106295C+2776588]
	GetHandleVerifier [0x00E52280+612144]
	(No symbol) [0x00D64F6C]
	(No symbol) [0x00D611D8]
	(No symbol) [0x00D612BB]
	(No symbol) [0x00D54857]
	BaseThreadInitThunk [0x75D67D59+25]
	RtlInitializeExceptionChain [0x7764B74B+107]
	RtlClearBits [0x7764B6CF+191]
